In [5]:
import numpy as np
import Elementos2D as BL4
import sympy as sp
import meshio
import Malha2D

In [ ]:

# #Entrada de Dados 
# #Nesses Dics devemos ter os nomes das propriedades iguais dos nossos Physical Groups
Materiais = {"MAT1":[1e9,0.3]}
#Nome do Apoio(igual do Physical Group:"direção a restringir o movimento(x,y ou engaste(ambos))")
apoios = {"Sx":"x","Sy":"y"}
#Carregamento em Face: "Nome da Carga(igual do Physical Group", direção:1= normal, 2 = tangencial,valor]
cargas = [("CARGA",1,1000)]

#Gera Malha
arquivo = "barraTriangulo.msh"
Malha2D.Gerar_barra_tracao(arquivo,L=1,h=0.1,nx=10,ny=10,triangulo=True)

#Retorna as propriedades
nn, XY, ne, IJ, MAT, ESP, na, AP, nc, P = Malha2D.Processar_malha2D(arquivo,Materiais,apoios,cargas,espessura=0.01)

# ##Entrada Manual
# ##Número de nós
# nn = 6
# XY = np.array([[0.0,0.0],[0.5,0.0],[1.0,0.0],[0.0,0.1],[0.5,0.1],[1.0,0.1]])
# ne = 4
# IJ = np.array([[1,2,5],[2,3,6],[1,5,4],[2,6,5]])

# MAT = np.array([[1e9,0.3],[1e9,0.3],[1e9,0.3],[1e9,0.3]])

# ESP = np.array([[0.01],[0.01],[0.01],[0.01]])

# na = 5
# AP = np.array([[1,1,0],[1,2,0],[2,2,0],[3,2,0],[4,1,0]])

# nc=1
# P = np.array([[2,2,1,1000]])


In [7]:
#Monta Rigidez Global
K_b = BL4.RigidezGlobal(nn,ne,MAT,ESP,XY,IJ)
#Monta Força Global
F_b = BL4.ForcapGlobal(nn,ESP,XY,IJ,nc,P)
#Aplica as CChs homogêneas
K_mod,F_mod = BL4.AplicaCCH(nn,na,AP,K_b,F_b)

#Calcula o Deslocamento
U = np.linalg.solve(K_mod,F_mod)

#Calcula a Tensão ao longo dos elementos
Sigma = BL4.CalculaTensaoMalha(nn,ne,MAT,ESP,XY,IJ,U)

#Calcula Tensão de Von Mises
TVM = BL4.TensaoVonMises2D(ne,Sigma)

#Calcula o Determinante do Jacobiano
DetJ = BL4.CalculaDetJ(ne,XY,IJ)

In [8]:
#Gera Malha com resultados
Malha2D.Exporta_para_Gmsh(arquivo,IJ, XY, U, TVM,DetJ)
Malha2D.AbreVisualizacaoGmsh(arquivo)